# Pandas — 15 ejemplos (con explicación en español)

Este cuaderno contiene **15 ejemplos** para aprender operaciones típicas en Pandas.
Se usan:
- datasets de `sklearn` (funcionan sin internet), y
- un CSV opcional desde la web (si falla la descarga, hay “fallback”).

Recomendación: antes de transformar datos, siempre inspecciona con `head()`, `info()` y `describe()`.


In [ ]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)


## Dataset helper

Estas funciones cargan datasets “seguros” (offline) y también permiten intentar descargar un CSV desde internet.


In [ ]:
from sklearn.datasets import load_iris, load_diabetes

def try_read_csv(url: str, **kwargs) -> pd.DataFrame:
    """Intenta leer un CSV desde la web; si falla, lanza un error claro."""
    try:
        return pd.read_csv(url, **kwargs)
    except Exception as e:
        raise RuntimeError(f"No se pudo descargar: {url}\nMotivo: {e}")

def iris_df() -> pd.DataFrame:
    iris = load_iris(as_frame=True)
    df = iris.frame.copy()
    df["species_name"] = df["target"].map(dict(enumerate(iris.target_names)))
    return df

def diabetes_df() -> pd.DataFrame:
    d = load_diabetes(as_frame=True)
    df = d.frame.copy()
    df.rename(columns={"target": "disease_progression"}, inplace=True)
    return df


## Example 1 — Crear Series y DataFrame

**Objetivo:** entender las dos estructuras principales de Pandas.

**Conceptos clave:**  
- `Series`: una columna con índice  
- `DataFrame`: tabla (varias columnas)

**Qué observar:** el índice y cómo se muestran las columnas.

**Qué deberías observar al ejecutar:**
- La salida (tabla).
- Cómo cambian los resultados al modificar parámetros.


In [ ]:
s = pd.Series([10, 20, 30], name="puntos")
df = pd.DataFrame({
    "nombre": ["Ana", "Ben", "Cesar", "Dina"],
    "edad": [18, 20, 19, 21],
    "calificacion": [88, 92, 79, 95],
})

display(s)
display(df)


## Example 2 — Leer un CSV desde la web (con fallback)

**Objetivo:** cargar datos reales con `read_csv` y mantener el notebook funcional sin internet.

**Conceptos clave:**  
- `pd.read_csv(url)`  
- manejo de excepciones  
- uso de dataset alternativo (Iris)

**Qué observar:** si estás offline, verás el dataset Iris como respaldo.

**Qué deberías observar al ejecutar:**
- La salida (tabla).
- Cómo cambian los resultados al modificar parámetros.


In [ ]:
url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/tips.csv"

try:
    tips = try_read_csv(url)
    print("Cargado desde la web:", url)
except RuntimeError as e:
    print(e)
    tips = iris_df()
    print("Usando dataset fallback: Iris")

display(tips.head())
print("shape:", tips.shape)


## Example 3 — Inspección rápida: head/tail/info/describe

**Objetivo:** obtener un “primer vistazo” de un dataset.

**Conceptos clave:**  
- `head()` y `tail()` para ver filas  
- `info()` para tipos y valores no nulos  
- `describe()` para estadísticas

**Qué observar:** diferencias entre columnas numéricas y categóricas.

**Qué deberías observar al ejecutar:**
- La salida (tabla).
- Cómo cambian los resultados al modificar parámetros.


In [ ]:
df = iris_df()
display(df.head())
display(df.tail())
df.info()  # tipos de datos y conteo de valores no nulos
display(df.describe(include="all"))


## Example 4 — Seleccionar con loc e iloc

**Objetivo:** seleccionar filas/columnas por posición o por etiqueta.

**Conceptos clave:**  
- `iloc`: posiciones (como listas)  
- `loc`: etiquetas (nombres de columnas/índices)

**Qué observar:** cómo cambia el resultado si cambias rangos/columnas.

**Qué deberías observar al ejecutar:**
- La salida (tabla).
- Cómo cambian los resultados al modificar parámetros.


In [ ]:
df = iris_df()

# Selección de columnas por nombre
display(df[["sepal length (cm)", "sepal width (cm)"]].head())

# Selección por posición: filas 0..4 y columnas 0..2
display(df.iloc[0:5, 0:3])

# Selección por etiqueta: filas 0..4 y columnas específicas
display(df.loc[0:4, ["petal length (cm)", "petal width (cm)", "species_name"]])


## Example 5 — Filtrado booleano con condiciones

**Objetivo:** filtrar registros como si fuera un `WHERE` en SQL.

**Conceptos clave:**  
- operadores `&` (AND) y `|` (OR)  
- `isin` para pertenencia

**Qué observar:** prueba distintas condiciones y cuenta filas.

**Qué deberías observar al ejecutar:**
- La salida (tabla).
- Cómo cambian los resultados al modificar parámetros.


In [ ]:
df = iris_df()
filtered = df[(df["sepal length (cm)"] > 5.5) & (df["species_name"].isin(["versicolor", "virginica"]))]  # AND con &
display(filtered.head())
print("Filas:", len(filtered))


## Example 6 — Ordenar con sort_values

**Objetivo:** ordenar por una o varias columnas.

**Conceptos clave:**  
- `sort_values(by=[...], ascending=[...])`

**Qué observar:** el orden por especie y dentro de especie por longitud de sépalo.

**Qué deberías observar al ejecutar:**
- La salida (tabla).
- Cómo cambian los resultados al modificar parámetros.


In [ ]:
df = iris_df()
sorted_df = df.sort_values(by=["species_name", "sepal length (cm)"], ascending=[True, False])
display(sorted_df.head(10))


## Example 7 — Crear/modificar columnas (vectorizado)

**Objetivo:** crear nuevas variables (features) sin bucles.

**Conceptos clave:**  
- operaciones entre columnas  
- uso de NumPy sobre columnas (`np.log`, etc.)

**Qué observar:** columnas nuevas y sus valores.

**Qué deberías observar al ejecutar:**
- La salida (tabla).
- Cómo cambian los resultados al modificar parámetros.


In [ ]:
df = iris_df()
df["sepal_area"] = df["sepal length (cm)"] * df["sepal width (cm)"]
df["petal_area"] = df["petal length (cm)"] * df["petal width (cm)"]
df["log_sepal_length"] = np.log(df["sepal length (cm)"])
display(df[["sepal_area", "petal_area", "log_sepal_length", "species_name"]].head())


## Example 8 — GroupBy y agregaciones

**Objetivo:** resumir datos por categoría.

**Conceptos clave:**  
- `groupby("col")`  
- `agg` con varias métricas

**Qué observar:** diferencias promedio entre especies.

**Qué deberías observar al ejecutar:**
- La salida (tabla).
- Cómo cambian los resultados al modificar parámetros.


In [ ]:
df = iris_df()
summary = (
    df.groupby("species_name")[["sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)"]]
      .agg(["mean", "std", "min", "max"])
)
display(summary)


## Example 9 — Pivot table (tabla dinámica)

**Objetivo:** responder preguntas agregadas rápidamente.

**Conceptos clave:**  
- `pd.pivot_table` con `values`, `index`, `aggfunc`

**Qué observar:** promedio/mediana/conteo por especie.

**Qué deberías observar al ejecutar:**
- La salida (tabla).
- Cómo cambian los resultados al modificar parámetros.


In [ ]:
df = iris_df()
pivot = pd.pivot_table(
    df,
    values="petal length (cm)",
    index="species_name",
    aggfunc=["mean", "median", "count"]
)
display(pivot)


## Example 10 — Merge / join

**Objetivo:** combinar tablas (similar a SQL JOIN).

**Conceptos clave:**  
- `merge` con clave `on=`  
- tipos de unión: `left`, `inner`, etc.

**Qué observar:** estudiantes sin registros de score quedan con NaN.

**Qué deberías observar al ejecutar:**
- La salida (tabla).
- Cómo cambian los resultados al modificar parámetros.


In [ ]:
students = pd.DataFrame({
    "student_id": [1, 2, 3, 4],
    "name": ["Ana", "Ben", "Cesar", "Dina"]
})

scores = pd.DataFrame({
    "student_id": [1, 2, 2, 4],
    "exam": ["midterm", "midterm", "final", "final"],
    "score": [88, 92, 94, 97]
})

merged = students.merge(scores, on="student_id", how="left")  # left join: mantiene todos los estudiantes
display(merged)


## Example 11 — Fechas y tiempos (datetime)

**Objetivo:** convertir strings a fechas y extraer partes útiles.

**Conceptos clave:**  
- `pd.to_datetime`  
- `.dt` para `day_name`, `hour`, etc.

**Qué observar:** columnas derivadas de la fecha.

**Qué deberías observar al ejecutar:**
- La salida (tabla).
- Cómo cambian los resultados al modificar parámetros.


In [ ]:
events = pd.DataFrame({
    "when": ["2025-01-05 10:30", "2025-01-06 14:15", "2025-02-01 09:00", "2025-02-05 18:40"],
    "event": ["Study", "Gym", "Study", "Movie"]
})
events["when"] = pd.to_datetime(events["when"])
events["date"] = events["when"].dt.date
events["day_name"] = events["when"].dt.day_name()
events["hour"] = events["when"].dt.hour
display(events)


## Example 12 — Valores faltantes: isna, fillna, dropna

**Objetivo:** detectar y tratar valores faltantes (`NaN`).

**Conceptos clave:**  
- `isna().sum()` para contar faltantes  
- `fillna` para imputar (rellenar)  
- `dropna` para eliminar filas según columnas críticas

**Qué observar:** cómo cambian los conteos después de imputar o eliminar.

**Qué deberías observar al ejecutar:**
- La salida (tabla).
- Cómo cambian los resultados al modificar parámetros.


In [ ]:
df = diabetes_df().copy()

# Crear faltantes artificiales (solo para el ejemplo)
df.loc[df.sample(frac=0.05, random_state=1).index, "bmi"] = np.nan
df.loc[df.sample(frac=0.05, random_state=2).index, "bp"] = np.nan

print("Conteo de NaN:")
display(df.isna().sum())

filled = df.copy()
filled["bmi"] = filled["bmi"].fillna(filled["bmi"].median())  # imputación con mediana
filled["bp"] = filled["bp"].fillna(filled["bp"].mean())       # imputación con media

print("Después de fillna:")
display(filled.isna().sum())

dropped = df.dropna(subset=["bmi", "bp"])
print("Filas después de dropna:", len(dropped))


## Example 13 — map y apply

**Objetivo:** transformar columnas y aplicar lógica por fila.

**Conceptos clave:**  
- `map` para traducciones/mapeos 1-a-1  
- `apply(axis=1)` para reglas por registro (fila)

**Qué observar:** etiquetas nuevas y su distribución.

**Qué deberías observar al ejecutar:**
- La salida (tabla).
- Cómo cambian los resultados al modificar parámetros.


In [ ]:
df = iris_df().copy()

# Mapear a etiquetas cortas
short = {"setosa": "S", "versicolor": "V", "virginica": "VG"}
df["species_short"] = df["species_name"].map(short)

# apply por fila: etiquetar tamaño de pétalo según reglas simples
def size_label(row):
    if row["petal length (cm)"] < 2:
        return "pequeño"
    elif row["petal length (cm)"] < 5:
        return "mediano"
    return "grande"

df["petal_size"] = df.apply(size_label, axis=1)
display(df[["species_name", "species_short", "petal length (cm)", "petal_size"]].head(12))


## Example 14 — Rolling window (ventana móvil)

**Objetivo:** calcular estadísticas en una ventana deslizante.

**Conceptos clave:**  
- `rolling(window=n).mean()` para media móvil  
- `rolling(...).std()` para volatilidad/dispersión local

**Qué observar:** primeros valores son NaN hasta completar la ventana.

**Qué deberías observar al ejecutar:**
- La salida (tabla).
- Cómo cambian los resultados al modificar parámetros.


In [ ]:
rng = np.random.default_rng(0)
ts = pd.Series(rng.normal(0, 1, size=50)).cumsum()  # serie acumulada
df = pd.DataFrame({"value": ts})
df["roll_mean_5"] = df["value"].rolling(window=5).mean()  # media móvil de 5 puntos
df["roll_std_5"] = df["value"].rolling(window=5).std()
display(df.head(10))


## Example 15 — MultiIndex y stack/unstack

**Objetivo:** pivotear entre formato “ancho” y “largo”.

**Conceptos clave:**  
- `pivot` para hacer columnas por categoría  
- `stack` convierte columnas a filas (formato largo)  
- `unstack` revierte el proceso

**Qué observar:** cómo cambia la estructura del índice.

**Qué deberías observar al ejecutar:**
- La salida (tabla).
- Cómo cambian los resultados al modificar parámetros.


In [ ]:
df = pd.DataFrame({
    "student": ["Ana", "Ana", "Ben", "Ben", "Cesar", "Cesar"],
    "exam": ["midterm", "final"] * 3,
    "score": [88, 91, 92, 94, 79, 85]
})

wide = df.pivot(index="student", columns="exam", values="score")
display(wide)

stacked = wide.stack()
print("Stacked (más largo):")
display(stacked)

unstacked = stacked.unstack()
print("Unstacked (vuelve a ancho):")
display(unstacked)
